In [1]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd
import numpy as np
#model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
model = SentenceTransformer('distiluse-base-multilingual-cased-v1')
#ns = pd.read_csv('sor_test.csv')
#ns = pd.read_excel('네이버_뉴스_제목_데이터.xlsx', engine='openpyxl')
#ns = ns.drop_duplicates(['Field2_text'], keep='first')
#text = ns['Field2_text'].tolist()
ns = pd.read_excel('네이버 뉴스_(10).xlsx', engine='openpyxl')
ns1 = pd.read_excel('네이버 뉴스_(11).xlsx', engine='openpyxl')
ns = ns.append(ns1, ignore_index = True)
ns = ns.fillna('')
ns = ns[ns['Text'] != '']
ns = ns.drop_duplicates(['Text'], keep='first')
ns.sort_values(by=['Text1'],inplace=True)
ns.reset_index(inplace=True)
text = ns['Text'].tolist()
ns

,index,Text,Text1
0,1173,바다가 일자리다…'2021 해양수산 온라인 취업박람회' 30일 개막,"""2021 해양수산 취업박람회"" 30일부터 온라인 진행"
1,1182,"일자리박람회 오프라인 면접, 다음 달 시작","""2021 해양수산 취업박람회"" 30일부터 온라인 진행"
2,1177,경북 온·오프라인 채용박람회 개최,"""2021 해양수산 취업박람회"" 30일부터 온라인 진행"
3,1176,강원일자리박람회 내달 2일 대면 면접 돌입,"""2021 해양수산 취업박람회"" 30일부터 온라인 진행"
4,1174,"안양시, 온·오프라인 연계 릴레이 일자리박람회","""2021 해양수산 취업박람회"" 30일부터 온라인 진행"
...,...,...,...
1713,906,희망회복자금 2차 신속지급 시작… 61만개社·1조원 규모,희망회복자금 2차 신속지급 시작
1714,903,희망회복자금 2차 신속지급 개시…61만여곳에 1조원 지원,희망회복자금 2차 신속지급 시작
1715,897,희망회복자금 2차 지급 시작…61.1만개사에 1조 지원,희망회복자금 2차 신속지급 시작
1716,923,희망회복자금 2차 신속지급 30일 개시…61만개사·1조원 규모,희망회복자금 2차 신속지급 시작


In [2]:
ns['tag_i'] = ''
ns.at[0, 'tag_i'] = 1
for i in ns.index:
    if i != 0:
        ns.at[i, 'tag_i'] = ns.loc[i - 1, 'tag_i']
        if ns.loc[i - 1, 'Text1'] != ns.loc[i, 'Text1']:
            ns.at[i, 'tag_i'] = ns.loc[i, 'tag_i'] + 1
tag = ns['tag_i'].tolist()

In [3]:
#Compute embeddings
embeddings = model.encode(text, convert_to_tensor=True)

#Compute cosine-similarities for each sentence with each other sentence
cosine_scores = util.pytorch_cos_sim(embeddings, embeddings)
cosine_scores = np.round(cosine_scores,4)

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer(max_features=500)
tfvect = vect.fit(text)

tfidv_df = pd.DataFrame(tfvect.transform(text).toarray(), columns = sorted(vect.vocabulary_))
tfidv_df

,100,100번째,10대,10월까지,12,12세,13명,1487명,16년,17세,...,확진자,활기,후텁지근,훼손,흉기로,흐리고,흥국생명,흥행,희망회복자금,희생
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1713,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.474294,0.0
1714,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.530311,0.0
1715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.499453,0.0
1716,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.461819,0.0


In [5]:
from sklearn.metrics.pairwise import cosine_similarity
tf_res = cosine_similarity(tfidv_df, tfidv_df)
tr_res = np.round(tf_res,4)

In [6]:
pairs1, pairs2, pairs3 = [],[],[]
for i in range(len(cosine_scores)):
    i_s = cosine_scores[i].tolist()
    i_s = {ii:score for ii, score in enumerate(i_s)}
    del i_s[i]
    i_s = sorted(i_s.items(), key=lambda x : x[1], reverse=True)    
    pairs1.append({'index': [i,i_s[0][0]], 'score':i_s[0][1] })
    
    i_tf = tf_res[i]
    i_tf = {ii:score for ii, score in enumerate(i_tf)}
    del i_tf[i]
    i_tf = sorted(i_tf.items(), key=lambda x : x[1], reverse=True)    
    pairs2.append({'index': [i,i_tf[0][0]], 'score':i_tf[0][1] })

In [7]:
def sens_cal(ns, idx, cut):
    t_df = pd.DataFrame()
    t_df['sco'] = cosine_scores[i].tolist()
    t_df['sco'] = t_df['sco'].apply(lambda x : x if x >= cut else float('inf'))
    t_df.at[idx, 'sco'] = float('inf')
    t_df['tag'] = ns['tag_i']
    # cut을 넘긴 텍스트 갯수
    c = len(t_df[t_df['sco'] != float('inf')])
    # cut을 넘긴 텍스트 중에 맞는것
    if c == 0:
        ct = 0
        pr = 0
    else:
        ct = len(t_df[(t_df['sco'] != float('inf')) & (t_df['tag'] == t_df.loc[idx, 'tag'])])
        pr = round(ct/c*100,2)
    # 같은 태그의 문장 개수
    tg1 = len(t_df[t_df['tag'] == t_df.loc[idx, 'tag']])
    #민감도 cut을 넘긴 텍스트중 tag가 맞는것/ 실제 tag에서 문장 개수
    #정밀도 cut을 넘긴 텍스트중 tag가 맞는것/ cut을 넘긴 텍스트 개수
    return [ct, tg1, round(ct/tg1*100,2) , ct, c, pr]
    
sen6,sen7,sen8  =[],[],[]
for i in range(len(cosine_scores)):
    sen6.append(sens_cal(ns, i, 0.6))
    sen7.append(sens_cal(ns, i, 0.7))
    sen8.append(sens_cal(ns, i, 0.8))

In [8]:
#pairs2 = sorted(pairs2, key=lambda x: x['score'], reverse=True)
tn_i, t1, t2 = [],[],[]
t_i, t_j1, t_j2 = [],[],[]
tn_i, tn_j1, tn_j2 = [],[],[]
ti_i, ti_j1, ti_j2 = [],[],[]
score1, score2, score3 = [], [], []
for pair in pairs1:
    i1, j1 = pair['index']
    
    t_i.append(text[i1])
    t1.append(text[j1])
     
    tn_i.append(i)
    tn_j1.append(j1)
    
    ti_i.append(tag[i1])    
    ti_j1.append(tag[j1])
    
    score1.append(np.array(pair['score']))

In [9]:
for pair in pairs2:
    i1, j1 = pair['index']
    t2.append(text[j1])
    tn_j2.append(j1)
    ti_j2.append(tag[j1])
    score2.append(np.array(pair['score']))

In [10]:
ff1 =pd.DataFrame()
ff1['text'] = t_i
ff1['text1'] = t1
ff1['text2'] = t2
#ff1['text3'] = t3
ff1['tag'] = ti_i
ff1['tag1'] = ti_j1
ff1['tag2'] = ti_j2
#ff1['tag3'] = ti_j3
ff1['score1'] = score1
ff1['score2'] = score2
#ff1['score3'] = score3
ff1['num_text'] = tn_i
ff1['num_text1'] = tn_j1
ff1['num_text2'] = tn_j2
#ff1['num_text3'] = tn_j3
sen6 = pd.DataFrame(sen6)
col6 = sen6.columns.tolist()
sen7 = pd.DataFrame(sen7)
col7 = sen7.columns.tolist()
sen8 = pd.DataFrame(sen8)
col8 = sen8.columns.tolist()

ff1['sensitivity(0.6)'] = sen6[col6[2]]
ff1['sensitivity(0.7)'] = sen7[col[2]]
ff1['sensitivity(0.8)'] = sen8[col8[2]]
ff1[' '] = ''
ff1['precision(0.6)'] = sen6[col6[5]]
ff1['precision(0.7)'] = sen7[col7[5]]
ff1['precision(0.8)'] = sen8[col8[5]]
ff1['  '] = ''
ff1['sensitivity(0.6)(True Tag)'] = sen6[col6[0]]
ff1['sensitivity(0.6)(#text each Tag)'] = sen6[col6[1]]
ff1['precision(0.6)(True Tag)'] = sen6[col6[3]]
ff1['precision(0.6)(above cutoff)'] = sen6[col6[4]]
ff1['sensitivity(0.7)(True Tag)'] = sen7[col[0]]
ff1['sensitivity(0.7)(#text each Tag)'] = sen7[col[1]]
ff1['precision(0.7)(True Tag)'] = sen7[col7[3]]
ff1['precision(0.7)(above cutoff)'] = sen7[col7[4]]
ff1['sensitivity(0.8)(True Tag)'] = sen8[col8[0]]
ff1['sensitivity(0.8)(#text each Tag)'] = sen8[col8[1]]
ff1['precision(0.8)(True Tag)'] = sen8[col8[3]]
ff1['precision(0.8)(above cutoff)'] = sen8[col8[4]]

ff1.to_csv('st_result4.csv', encoding='utf-8-sig')

In [11]:
print('1:Sentence_transformers:',len(ff1[ff1['tag']== ff1['tag1']]),'/',len(ff1),
      ', 정확도: ', round(len(ff1[ff1['tag']== ff1['tag1']])/len(ff1)*100,2) ,'%')
print('2:TF-IDF:               ',len(ff1[ff1['tag']== ff1['tag2']]),'/',len(ff1),
      ', 정확도: ', round(len(ff1[ff1['tag']== ff1['tag2']])/len(ff1)*100,2) ,'%')

1:Sentence_transformers: 1352 / 1718 , 정확도:  78.7 %
2:TF-IDF:                1166 / 1718 , 정확도:  67.87 %
